In [35]:
import re

import pandas as pd
import numpy as np

from tqdm.notebook import tqdm
tqdm.pandas()

In [42]:
df = pd.read_csv('../data/abbr_term.csv', sep='\t')

In [43]:
from abbr_decoder import AbbreviationDecoder


def load_extractor():
    import sys
  
    # setting path
    sys.path.append('../abbr_extraction/')
    
    from abbreviation_extractor import load_extractor
    
    sys.path.append('../abbr_decoding/')
    
    return load_extractor()

In [44]:
decoder = AbbreviationDecoder()
extractor = load_extractor()

In [55]:
abbr2term = {abbr.lower(): term for abbr, term in df[['abbreviation', 'term']].iloc if not pd.isna(abbr)}

In [58]:
abbr2term['ад'] = 'артериальное давление'

In [61]:
resolved_pairs = {}

def resolve_abbreviations(text, window=5):
    new_text = []
    words = text.split() + [''] * window
    current_context = [''] * window + words[:window]
    
    for i in range(window, len(words)):
        word = current_context[window]
        
        if extractor.is_abbreviation(word) and extractor._abbr_predictor.predict(word):
            resolved = decoder.predict(word, ' '.join(current_context))[0]
            couldnt_find = (resolved == word)
            
            if couldnt_find:
                clean_word = re.sub('[^а-яА-Яa-zA-Z]', '', word)
                resolved = abbr2term.get(clean_word.lower(), word)
                
            resolved_pairs[word] = (resolved, couldnt_find)
            
            current_context[window] = resolved
            
        new_text.append(current_context.pop(0))
        current_context.append(words[i])
    
    new_text.extend(current_context)
    return " ".join(new_text).strip()

In [62]:
text = 'Постепенное расширение пищевого рациона с увеличением объема жидкой пищи и овощей. Избегать жирной, острой пищи Есть до 6 раз в день мелкими порциями Наблюдение кардиолога и гастроэнтеролога ПРОДОЛЖИТЬ ПОСТОЯННЫЙ ПРИЕМ:  ДИОВАН 80 МГ УТРОМ И ВЕЧЕРОМ ДИЛАТРЕНД 6,25 МГ УТРОМ И ВЕЧЕРОМ АРИФОН-РЕТАРД 1,5 МГ УТРОМ   ПАРИЕТ 20 МГ ЗА 30 МИНУТ ДО ЗАВТРАКА 1 НЕДЕЛЮ, ЗАТЕМ ПО 10 МГ ЗА 30 МИНУТ ДО ЗАВТРАКА 2 МЕСЯЦА  ДЕ-НОЛ ЧЕРЕЗ 1 ЧАС ПОСЛЕ ЗАВТРАКА, ОБЕДА, УЖИНА И НА НОЧЬ 1 МЕСЯЦ  КРЕОН 10000 ЕД В КАЖДЫЙ ПРИЕМ ПИЩИ, А ЕСЛИ УПОТРЕБЛЯЕТСЯ БЕЛКОВАЯ ПИЩА (МЯСО, РЫБА, ТВОРОГ, ЯЙЦО), ТО 25000 ЕД (СУММАРНО ЗА СУТКИ 80000-100000 ЕД)  ПРИ ВЗДУТИИ ЖИВОТА ПЕКСАН-R  '

In [63]:
resolve_abbreviations(text)

'Постепенное расширение пищевого рациона с увеличением объема жидкой пищи и овощей. Избегать жирной, острой пищи Есть до 6 раз в день мелкими порциями Наблюдение кардиолога и гастроэнтеролога ПРОДОЛЖИТЬ ПОСТОЯННЫЙ ПРИЕМ: ДИОВАН 80 миллиграмм УТРОМ И ВЕЧЕРОМ ДИЛАТРЕНД 6,25 миллиграмм УТРОМ И ВЕЧЕРОМ АРИФОН-РЕТАРД 1,5 миллиграмм УТРОМ ПАРИЕТ 20 миллиграмм ЗА 30 МИНУТ ДО ЗАВТРАКА 1 НЕДЕЛЮ, ЗАТЕМ ПО 10 миллиграмм ЗА 30 МИНУТ ДО ЗАВТРАКА 2 МЕСЯЦА ДЕ-НОЛ ЧЕРЕЗ 1 ЧАС ПОСЛЕ ЗАВТРАКА, ОБЕДА, УЖИНА И НА НОЧЬ 1 МЕСЯЦ КРЕОН 10000 единицы В КАЖДЫЙ ПРИЕМ ПИЩИ, А ЕСЛИ УПОТРЕБЛЯЕТСЯ БЕЛКОВАЯ ПИЩА (МЯСО, РЫБА, ТВОРОГ, ЯЙЦО), ТО 25000 единицы (СУММАРНО ЗА СУТКИ 80000-100000 единицы ПРИ ВЗДУТИИ ЖИВОТА ПЕКСАН-R'

## Evaulation

In [64]:
anamnesises_df = pd.read_csv(
    '../../ISC_Analys/all_anamnesises.txt', 
    encoding='cp1251', 
    sep='\t', 
    usecols=['Diagnosis', 'RecordEMC']
).dropna()

In [65]:
anamnesises_df['Letter'] = anamnesises_df.Diagnosis.str[0]

In [66]:
def clear_tag(text):
    return re.sub('<[^>]*>', ' ', text)

In [67]:
anamnesises_df.RecordEMC = anamnesises_df.RecordEMC.map(clear_tag)

In [68]:
anamnesises_df['RecordResolvedAbbreviations'] = anamnesises_df.RecordEMC.progress_apply(resolve_abbreviations)

  0%|          | 0/7006 [00:00<?, ?it/s]

In [69]:
from normalizer import normalize_sentences

In [70]:
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [71]:
abbrs = set(pd.read_csv('../data/abbr_term.csv', sep='\t')['abbreviation'])

In [72]:
normalized_abbrs_text = normalize_sentences(anamnesises_df.RecordResolvedAbbreviations, except_words=abbrs)

In [73]:
normalized_text = normalize_sentences(anamnesises_df.RecordEMC, except_words=abbrs)

In [74]:
norm_abbr = [" ".join([" ".join(word) for word in text]) for text in normalized_abbrs_text]
norm_text = [" ".join([" ".join(word) for word in text]) for text in normalized_text]

In [75]:
anamnesises_df.RecordEMC.iloc[2]

'    Пациент был госпитализирован с клиникой нестабильной стенокардии, прогрессирующей стенокардии с 20.03.08г., на фоне дислипидемии, сахарного диабета, курения. Из анамнеза известно, что впервые боли в грудной клетке возникли в январе 2008г., лечился в стационаре по месту жительства, отмечал некоторую стабилизацию состояния, но с 20.03.08г. вновь рецидивировали приступы ангинозных болей, снизилась толерантность к физ. нагрузкам, обратился в ФГУ ФЦСКЭ: 01.04.08г.- проведена коронарография, выявлен: эксцентричный стеноз до субокклюзии с признаками интракоронарного тромбоза в огибающей артерии. Одномоментно проведена операция коронарной ангиопластики со стентированием огибающей артерии (стент braun melsungen). В дальнейшем, состояние пациента оставалось стабильным, ангинозные боли не рецидивировали. Ежегодно проводилось обследование: стрессЭХОКГ ежегодно тесты отрицательные. Клиники стенокардии не описывает. Несколько нарастает одышка при большой ФН. По данным стрессЭХОКГ мощность 13,5 

In [76]:
anamnesises_df.RecordResolvedAbbreviations.iloc[2]

'Пациент был госпитализирован с клиникой нестабильной стенокардии, прогрессирующей стенокардии с 20.03.08г., на фоне дислипидемии, сахарного диабета, курения. Из анамнеза известно, что впервые боли в грудной клетке возникли в январе 2008г., лечился в стационаре по месту жительства, отмечал некоторую стабилизацию состояния, но с 20.03.08г. вновь рецидивировали приступы ангинозных болей, снизилась толерантность к физ. нагрузкам, обратился в ФГУ ФЦСКЭ: 01.04.08г.- проведена коронарография, выявлен: эксцентричный стеноз до субокклюзии с признаками интракоронарного тромбоза в огибающей артерии. Одномоментно проведена операция коронарной ангиопластики со стентированием огибающей артерии (стент braun melsungen). В дальнейшем, состояние пациента оставалось стабильным, ангинозные боли не рецидивировали. Ежегодно проводилось обследование: стрессЭХОКГ ежегодно тесты отрицательные. Клиники стенокардии не описывает. Несколько нарастает одышка при большой фебрильная нейтропения По данным стрессЭХОКГ

In [77]:
abbr_vectorizer = TfidfVectorizer(max_features=400)
features_abbr = abbr_vectorizer.fit_transform(norm_abbr)

text_vectorizer = TfidfVectorizer(max_features=400)
features_text = text_vectorizer.fit_transform(norm_text)

In [78]:
X_train_abbr, X_test_abbr, X_train_text, X_test_text, y_train, y_test = train_test_split(
    features_abbr,
    features_text,
    anamnesises_df['Letter'],
    test_size=0.3,
    random_state=42
)

In [79]:
def evalulate_model(X_train, X_test):
    clf = LogisticRegression()
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    print('f1_score:', f1_score(y_pred, y_test, average='weighted'))
    print('accuracy:', clf.score(X_test, y_test))

In [80]:
evalulate_model(X_train_abbr, X_test_abbr)

f1_score: 0.8755635750532718
accuracy: 0.8158896289248335


In [81]:
evalulate_model(X_train_text, X_test_text)

f1_score: 0.8762913924083476
accuracy: 0.8168411037107517


- Классификатор на текстах клиинческих рекомендаций - код мкб, заболевание

In [84]:
len(resolved_pairs)

2062